In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
import os
import math
from collections import Counter
from torch.utils.data import DataLoader, Sampler
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torchmetrics.functional.text import bleu_score
import spacy

# Load Spacy
spacy_eng = spacy.load("en_core_web_sm")
spacy_fr = spacy.load("fr_core_news_sm")

In [6]:

# --- 1. VOCABULARY CLASS ---
class Vocabulary:
    def __init__(self, freq_threshold=2, max_size=80000):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold
        self.max_size = max_size

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    @staticmethod
    def tokenizer_fr(text):
        return [tok.text.lower() for tok in spacy_fr.tokenizer(text)]

    def build_vocabulary(self, sentence_list, tokenizer):
        frequencies = Counter()
        idx = 4
        for sentence in sentence_list:
            for word in tokenizer(sentence):
                frequencies[word] += 1
        common_words = frequencies.most_common(self.max_size - 4)
        for word, count in common_words:
            if count >= self.freq_threshold:
                self.stoi[word] = idx
                self.itos[idx] = word
                idx += 1

    def numericalize(self, text, tokenizer):
        tokenized_text = tokenizer(text)
        return [self.stoi[token] if token in self.stoi else self.stoi["<UNK>"] for token in tokenized_text]

# --- 2. DATASET CLASS (WITH REVERSING) ---
class WMT14Dataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, source_vocab, target_vocab):
        self.hf_dataset = hf_dataset
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, index):
        pair = self.hf_dataset[index]['translation']
        src_text = pair['en']
        trg_text = pair['fr']

        # Numericalize and REVERSE Source
        src_indices = self.source_vocab.numericalize(src_text, self.source_vocab.tokenizer_eng)
        src_indices = src_indices[::-1] # <--- REVERSING INPUT HERE
        
        numericalized_source = [self.source_vocab.stoi["<SOS>"]] + src_indices + [self.source_vocab.stoi["<EOS>"]]
        numericalized_target = [self.target_vocab.stoi["<SOS>"]] + \
                               self.target_vocab.numericalize(trg_text, self.target_vocab.tokenizer_fr) + \
                               [self.target_vocab.stoi["<EOS>"]]

        return torch.tensor(numericalized_source), torch.tensor(numericalized_target)

class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
    def __call__(self, batch):
        source = [item[0] for item in batch]
        target = [item[1] for item in batch]
        source = pad_sequence(source, batch_first=False, padding_value=self.pad_idx)
        target = pad_sequence(target, batch_first=False, padding_value=self.pad_idx)
        return source, target

# --- 3. MODEL ARCHITECTURE ---
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.lstm(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input = trg[0, :]
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            top1 = output.argmax(1)
            input = trg[t] if random.random() < teacher_forcing_ratio else top1
        return outputs

def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [7]:
class BucketBatchSampler(Sampler):
    def __init__(self, dataset, batch_size, noise=0.1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.noise = noise # Adds a little randomness to the sorting so batches aren't identical every epoch
        
        # 1. Pre-calculate lengths of all source sentences
        # (This takes a moment at startup but saves huge time during training)
        print("Calculating dataset lengths for bucketing...")
        self.lengths = []
        for i in range(len(dataset)):
            # Access the raw text directly from the HF dataset to be fast
            src_text = dataset.hf_dataset[i]['translation']['en']
            self.lengths.append(len(src_text)) # approximate length by char count (faster) or use tokenizer
            
    def __iter__(self):
        # 2. Create indices and add noise to lengths
        # We add random noise to lengths so the sort order changes slightly every epoch
        indices = list(range(len(self.dataset)))
        
        # Zip indices with their noisy lengths
        noisy_lengths = [l + random.uniform(-self.noise, self.noise) for l in self.lengths]
        indices_with_lens = list(zip(indices, noisy_lengths))
        
        # 3. Sort by length
        indices_with_lens.sort(key=lambda x: x[1])
        sorted_indices = [x[0] for x in indices_with_lens]
        
        # 4. Create batches
        # Since the list is sorted, chunks of 'batch_size' will naturally have similar lengths
        batches = [sorted_indices[i:i + self.batch_size] for i in range(0, len(sorted_indices), self.batch_size)]
        
        # 5. Shuffle the BATCHES (Important!)
        # We want to train on similar-length sentences together, 
        # BUT we don't want to always train on short sentences first and long ones last.
        random.shuffle(batches)
        
        # Yield one batch at a time
        for batch in batches:
            yield batch

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [8]:
# --- CONFIGURATION ---
BATCH_SIZE = 32
SUBSET_SIZE = 10000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using Device: {DEVICE}")

# 1. Load Data
dataset = load_dataset("wmt14", "fr-en")
train_subset = dataset['train'].select(range(SUBSET_SIZE))
valid_subset = dataset['validation'].select(range(1000))

# 2. Build Vocab
print("Building Vocabularies...")
vocab_en = Vocabulary(freq_threshold=1, max_size=80000)
vocab_en.build_vocabulary([item['translation']['en'] for item in train_subset], vocab_en.tokenizer_eng)

vocab_fr = Vocabulary(freq_threshold=1, max_size=80000)
vocab_fr.build_vocabulary([item['translation']['fr'] for item in train_subset], vocab_fr.tokenizer_fr)

print(f"Unique English Words: {len(vocab_en)}")
print(f"Unique French Words: {len(vocab_fr)}")

# 3. DataLoaders
pad_idx = vocab_en.stoi["<PAD>"]

# Create the Dataset (No change here)
train_dataset_obj = WMT14Dataset(train_subset, vocab_en, vocab_fr)

# Initialize the Bucket Sampler
# This replaces the 'shuffle=True' standard logic
bucket_sampler = BucketBatchSampler(train_dataset_obj, batch_size=BATCH_SIZE)

# Create the DataLoader
train_loader = DataLoader(
    train_dataset_obj,
    batch_sampler=bucket_sampler, # <--- Pass the custom sampler here
    num_workers=0,
    collate_fn=MyCollate(pad_idx)
)

print("Optimized DataLoader ready.")

Using Device: mps


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

Building Vocabularies...
Unique English Words: 10731
Unique French Words: 14472
Calculating dataset lengths for bucketing...
Optimized DataLoader ready.


In [9]:
# --- HYPERPARAMETERS ---
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_fr)
ENC_EMB_DIM = 1000
DEC_EMB_DIM = 1000
HID_DIM = 1000
N_LAYERS = 4
DROPOUT = 0.2
N_EPOCHS = 20 
CLIP = 5

SEEDS = [1, 2, 3, 4, 5]

def train_one_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        src, trg = src.to(DEVICE), trg.to(DEVICE)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# --- TRAINING LOOP FOR 5 MODELS ---
for seed in SEEDS:
    print(f"\n{'='*20}")
    print(f"TRAINING MODEL {seed}/5 (Seed: {seed})")
    print(f"{'='*20}")
    
    # 1. Set Seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available(): torch.mps.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed(seed)
    
    # 2. Initialize Fresh Model
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
    model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)
    model.apply(init_weights) # Paper initialization
    
    # 3. Optimizer & Criterion
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
    
    # 4. Train
    for epoch in range(N_EPOCHS):
        loss = train_one_epoch(model, train_loader, optimizer, criterion, CLIP)
        print(f"Seed {seed} | Epoch {epoch+1} | Loss: {loss:.3f} | PPL: {math.exp(loss):.2f}")
        
    # 5. Save Model
    save_path = f"ensembel_model_seed_{seed}.pt"
    torch.save(model.state_dict(), save_path)
    print(f"--> Saved {save_path}")

print("\nAll 5 models trained and saved successfully.")


TRAINING MODEL 1/5 (Seed: 1)
Seed 1 | Epoch 1 | Loss: 6.406 | PPL: 605.33
Seed 1 | Epoch 2 | Loss: 6.167 | PPL: 476.58
Seed 1 | Epoch 3 | Loss: 6.074 | PPL: 434.44
Seed 1 | Epoch 4 | Loss: 5.847 | PPL: 346.13
Seed 1 | Epoch 5 | Loss: 5.607 | PPL: 272.27
Seed 1 | Epoch 6 | Loss: 5.426 | PPL: 227.22
Seed 1 | Epoch 7 | Loss: 5.279 | PPL: 196.23
Seed 1 | Epoch 8 | Loss: 5.150 | PPL: 172.37
Seed 1 | Epoch 9 | Loss: 5.051 | PPL: 156.11
Seed 1 | Epoch 10 | Loss: 4.962 | PPL: 142.90
Seed 1 | Epoch 11 | Loss: 4.877 | PPL: 131.19
Seed 1 | Epoch 12 | Loss: 4.771 | PPL: 117.98
Seed 1 | Epoch 13 | Loss: 4.698 | PPL: 109.69
Seed 1 | Epoch 14 | Loss: 4.644 | PPL: 103.96
Seed 1 | Epoch 15 | Loss: 4.586 | PPL: 98.12
Seed 1 | Epoch 16 | Loss: 4.503 | PPL: 90.27
Seed 1 | Epoch 17 | Loss: 4.454 | PPL: 86.00
Seed 1 | Epoch 18 | Loss: 4.375 | PPL: 79.42
Seed 1 | Epoch 19 | Loss: 4.308 | PPL: 74.32
Seed 1 | Epoch 20 | Loss: 4.258 | PPL: 70.70
--> Saved ensembel_model_seed_1.pt

TRAINING MODEL 2/5 (Seed: 2)


In [17]:
def beam_search_decode_ensemble(models_list, sentence, vocab_src, vocab_trg, beam_size=2, max_len=50, device='cpu'):
    # Ensure models are in eval mode
    for m in models_list: m.eval()
    
    # Prepare Input
    if isinstance(sentence, str): tokens = vocab_src.tokenizer_eng(sentence)
    else: tokens = [token.lower() for token in sentence]
    
    indices = [vocab_src.stoi.get(t, vocab_src.stoi["<UNK>"]) for t in tokens]
    indices = indices[::-1] # REVERSE
    indices = [vocab_src.stoi["<SOS>"]] + indices + [vocab_src.stoi["<EOS>"]]
    src_tensor = torch.LongTensor(indices).unsqueeze(1).to(device)

    with torch.no_grad():
        # Encoder states for all models
        initial_states = []
        for m in models_list:
            h, c = m.encoder(src_tensor)
            initial_states.append((h, c))

        hypotheses = [(0.0, [vocab_trg.stoi["<SOS>"]], initial_states)]

        for _ in range(max_len):
            all_candidates = []
            for score, seq, states_list in hypotheses:
                if seq[-1] == vocab_trg.stoi["<EOS>"]:
                    all_candidates.append((score, seq, states_list))
                    continue
                
                last_token = torch.LongTensor([seq[-1]]).to(device)
                
                # Run all models and AVERAGE logits
                avg_log_probs = None
                new_states_list = []
                for i, m in enumerate(models_list):
                    prev_h, prev_c = states_list[i]
                    pred, new_h, new_c = m.decoder(last_token, prev_h, prev_c)
                    new_states_list.append((new_h, new_c))
                    
                    # --- FIX START ---
                    # Squeeze the batch dimension: [1, vocab_size] -> [vocab_size]
                    pred = pred.squeeze(0)
                    # --- FIX END ---

                    log_probs = F.log_softmax(pred, dim=0)
                    if avg_log_probs is None: avg_log_probs = log_probs
                    else: avg_log_probs += log_probs
                
                avg_log_probs = avg_log_probs / len(models_list)
                top_k_probs, top_k_ids = avg_log_probs.topk(beam_size * 2)

                for i in range(len(top_k_ids)):
                    all_candidates.append((score + top_k_probs[i].item(), seq + [top_k_ids[i].item()], new_states_list))

            hypotheses = sorted(all_candidates, key=lambda x: x[0], reverse=True)[:beam_size]
            if all(h[1][-1] == vocab_trg.stoi["<EOS>"] for h in hypotheses): break

    best_seq = hypotheses[0][1]
    decoded_words = [vocab_trg.itos[idx] for idx in best_seq]
    if "<SOS>" in decoded_words: decoded_words.remove("<SOS>")
    if "<EOS>" in decoded_words: decoded_words = decoded_words[:decoded_words.index("<EOS>")]
    return " ".join(decoded_words)

# --- BLEU EVALUATOR ---
def evaluate_ensemble(data_subset, models_list, vocab_src, vocab_trg, device, beam_size, n_gram=4):
    targets = []
    predictions = []
    print(f"Evaluating Ensemble on {len(data_subset)} samples...")
    
    for i, datum in enumerate(data_subset):
        if (i+1)%50 == 0: print(f"Processed {i+1}...")
        pair = datum['translation']
        pred = beam_search_decode_ensemble(     
                models_list, 
                pair['en'], 
                vocab_src, 
                vocab_trg, 
                beam_size, 
                max_len=50, # Optional: Explicitly set max_len here if you want
                device=device
            )
        targets.append([pair['fr']])
        predictions.append(pred)
        
    return bleu_score(predictions, targets, n_gram=n_gram).item() * 100

In [19]:
# Load the 5 Saved Models
ensemble_models = []
print("Loading Ensemble Models...")

for seed in SEEDS:
    # Re-instantiate architecture
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
    m = Seq2Seq(enc, dec, DEVICE).to(DEVICE)
    
    # Load weights
    m.load_state_dict(torch.load(f"ensembel_model_seed_{seed}.pt", weights_only=True))
    ensemble_models.append(m)
    print(f"Loaded model_seed_{seed}.pt")

# Sanity Check
print("\n--- Manual Ensemble Check ---")
test_sen = "the cat sat on the mat"
print(f"Input: {test_sen}")
print(f"Output: {beam_search_decode_ensemble(ensemble_models, test_sen, vocab_en, vocab_fr, beam_size=2, device=DEVICE)}")


Loading Ensemble Models...
Loaded model_seed_1.pt
Loaded model_seed_2.pt
Loaded model_seed_3.pt
Loaded model_seed_4.pt
Loaded model_seed_5.pt

--- Manual Ensemble Check ---
Input: the cat sat on the mat
Output: le applaudissements de la


In [18]:

# Final BLEU Score
n_grams_l = [1, 2, 3, 4]
for _,n in enumerate(n_grams_l):
    final_score = evaluate_ensemble(valid_subset, ensemble_models, vocab_en, vocab_fr, DEVICE, beam_size=2, n_gram=n)
    print(f"\n=================================")
    print(f"FINAL ENSEMBLE BLEU SCORE with n_gram={n}: {final_score:.2f}")
    print(f"=================================")

Evaluating Ensemble on 1000 samples...
Processed 50...
Processed 100...
Processed 150...
Processed 200...
Processed 250...
Processed 300...
Processed 350...
Processed 400...
Processed 450...
Processed 500...
Processed 550...
Processed 600...
Processed 650...
Processed 700...
Processed 750...
Processed 800...
Processed 850...
Processed 900...
Processed 950...
Processed 1000...

FINAL ENSEMBLE BLEU SCORE with n_gram=1: 9.23
Evaluating Ensemble on 1000 samples...
Processed 50...
Processed 100...
Processed 150...
Processed 200...
Processed 250...
Processed 300...
Processed 350...
Processed 400...
Processed 450...
Processed 500...
Processed 550...
Processed 600...
Processed 650...
Processed 700...
Processed 750...
Processed 800...
Processed 850...
Processed 900...
Processed 950...
Processed 1000...

FINAL ENSEMBLE BLEU SCORE with n_gram=2: 2.76
Evaluating Ensemble on 1000 samples...
Processed 50...
Processed 100...
Processed 150...
Processed 200...
Processed 250...
Processed 300...
Processe